In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [2]:
class Value:
    def __init__(self,data,_children=(),_op='',label=''):
        self.data=data
        self.grad=0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label=label
        
    def __repr__(self):
        return f"Values(data={self.data})"
    
    def __add__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.data + other.data,(self,other),'+')
        def _backward():
            self.grad += 1.0*out.grad
            other.grad += 1.0*out.grad
        out._backward = _backward
        return out

    def __mul__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.data * other.data,(self,other),'*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad 
        out._backward = _backward
        return out
    
    def __pow__(self,other):
        assert isinstance(other,(int,float)), "only supports integers,floats"
        out=Value(self.data**other,(self,),f'**{other}')

        def _backward():
            self.grad += (other)*self.data**(other-1)*out.grad
        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
    
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

    def __neg__(self):
        return self * -1
    
    def __rmul__(self,other):
        return self*other

    def __truediv__(self,other):
        return self * other**-1
    
    

    def __radd__(self,other):
        return self + other
    
    def __sub__(self,other):
        return self + (-other)

    def __rsub__(self,other):
        return other + (-self)

    def tanh(self):
        x=self.data
        t=(math.exp(2*x)-1)/(math.exp(2*x)+1)
        out = Value(t,(self,),'tanh')
        def _backward():
            self.grad += (1-t**2)*out.grad
        out._backward = _backward
        return out
    
    def exp(self):
        x=self.data
        out = Value(math.exp(x),(self,),'exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out
    
    

In [3]:
from graphviz import Digraph
def trace(root):
    nodes,edges = set(),set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child,v))
                build(child)
    
    build(root)
    return nodes,edges


def draw_dot(root):
    dot = Digraph(format='svg',graph_attr={'rankdir':'LR'})

    nodes,edges = trace(root)
    for n in nodes:
        uid=str(id(n))
        dot.node(name=uid,label="{ %s | data %.4f | grad %.4f }" % (n.label,n.data,n.grad),shape='record')
        if n._op:
            dot.node(name=uid + n._op,label=n._op)
            dot.edge(uid+n._op,uid)

    for n1,n2 in edges:
        dot.edge(str(id(n1)),str(id(n2))+n2._op)
    return dot

In [4]:
class Neuron:
    def __init__(self,nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))

    def __call__(self,x):
        act = sum((wi*xi for wi,xi in zip(self.w,x)),self.b)
        out = act.tanh()
        return out

    def parameters(self):
        return self.w + [self.b]
    
class Layer:
    def __init__(self,nin,nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self,x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs)==1 else outs

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
    
class MLP:
    def __init__(self,nin,nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i],sz[i+1]) for i in range(len(nouts))]

    def __call__(self,x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [11]:
xs = [
    [2.0,3.0,-1.0],
    [3.0,-1.0,0.5],
    [0.5,1.0,1.0],
    [1.0,1.0,-1.0],
]
ys = [1.0,-1.0,1.0,-1.0]
ypred = [n(x) for x in xs]
ypred

[Values(data=0.9480042816519226),
 Values(data=-0.9906806215311441),
 Values(data=0.9492824917592777),
 Values(data=-0.9334666029275056)]

In [12]:
for k in range(20):
    ypred = [n(x) for x in xs]
    loss = sum((yout-ygt)**2 for ygt,yout in zip(ys,ypred))
    
    for p in n.parameters():
        p.grad=0.0
    loss.backward()


    for p in n.parameters():
        p.data += -0.05 * p.grad

    print(k,loss.data)

0 0.009789364109732303
1 0.009600741189166415
2 0.009419001272378574
3 0.009243780746124898
4 0.009074740892634256
5 0.008911565810856673
6 0.00875396054139402
7 0.00860164937230877
8 0.008454374305866546
9 0.008311893668735228
10 0.008173980850289538
11 0.00804042315551381
12 0.007911020760592807
13 0.007785585760669362
14 0.007663941300457636
15 0.0075459207794570035
16 0.007431367124434379
17 0.007320132122653318
18 0.007212075810037344
19 0.007107065909080504


In [13]:
ypred

[Values(data=0.9553410980134124),
 Values(data=-0.9918086265306277),
 Values(data=0.9570310144423076),
 Values(data=-0.943438387016803)]